# RAG model 
### PDF-DATA ingestion to vector DB

In [10]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [11]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 1 PDF files to process

Processing: Statistic_For_Data_Science_1704226429.pdf
  ✓ Loaded 38 pages

Total documents loaded: 38


In [12]:
all_pdf_documents

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': '..\\data\\pdf\\Statistic_For_Data_Science_1704226429.pdf', 'total_pages': 38, 'page': 0, 'page_label': '1', 'source_file': 'Statistic_For_Data_Science_1704226429.pdf', 'file_type': 'pdf'}, page_content='Page 1 \n \nSH                                                       Statistics for Data Science \n \n \nStatistics For Data Science \n \nWhat is Data? \n According to the Oxford “Data is distinct pieces of information, usually \nformatted in a special way”. \nData is measured, collected and reported, and analysed, whereupon it is often \nvisualized using graphs, images or other analysis tools. Raw data may be a collection of \nnumbers or characters before it’s been cleaned and corrected by researchers. \n \nWhat is data science?  \n Data science is the study of data It involves developing methods of recording, \nstoring, and analysing data to effectively extract useful information. The goal of d

In [13]:
# text splitting into chunks
def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [16]:
chunks=split_documents(all_pdf_documents)
chunks

Split 38 documents into 91 chunks

Example chunk:
Content: Page 1 
 
SH                                                       Statistics for Data Science 
 
 
Statistics For Data Science 
 
What is Data? 
 According to the Oxford “Data is distinct pieces of i...
Metadata: {'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': '..\\data\\pdf\\Statistic_For_Data_Science_1704226429.pdf', 'total_pages': 38, 'page': 0, 'page_label': '1', 'source_file': 'Statistic_For_Data_Science_1704226429.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': '..\\data\\pdf\\Statistic_For_Data_Science_1704226429.pdf', 'total_pages': 38, 'page': 0, 'page_label': '1', 'source_file': 'Statistic_For_Data_Science_1704226429.pdf', 'file_type': 'pdf'}, page_content='Page 1 \n \nSH                                                       Statistics for Data Science \n \n \nStatistics For Data Science \n \nWhat is Data? \n According to the Oxford “Data is distinct pieces of information, usually \nformatted in a special way”. \nData is measured, collected and reported, and analysed, whereupon it is often \nvisualized using graphs, images or other analysis tools. Raw data may be a collection of \nnumbers or characters before it’s been cleaned and corrected by researchers. \n \nWhat is data science?  \n Data science is the study of data It involves developing methods of recording, \nstoring, and analysing data to effectively extract useful information. The goal of d

# Embedding And VectorStoreDB

In [17]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

e:\Gen AI Course\RAG Crash course\rven\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):   #hugging face model for embeddings
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2


e:\Gen AI Course\RAG Crash course\rven\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vikrant\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling 

Model loaded successfully. Embedding dimension: 384


# Vectorstore

In [19]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [ ]:
# converting the text to embeddings
texts=[doc.page_content for doc in chunks]

# generating embeddings
embeddings=embedding_manager.generate_embeddings(texts)

# store in the vector DB
vectorstore.add_documents(chunks,embeddings) 

Generating embeddings for 91 texts...


Batches: 100%|██████████| 3/3 [00:01<00:00,  1.56it/s]


Generated embeddings with shape: (91, 384)
Adding 91 documents to vector store...
Successfully added 91 documents to vector store
Total documents in collection: 91


# Retriever Pipeline from VectorStoreDB

In [25]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)



In [29]:
rag_retriever.retrieve("What is Normal distribution of the graph")

Retrieving documents for query: 'What is Normal distribution of the graph'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 76.18it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


[{'id': 'doc_3c6173b5_90',
  'content': 'Page 38 \n \nSH                                                       Statistics for Data Science \n \nA Normal distribution curve has the following properties: \no Symmetrical around its mean value \no Mean = Median = Mode \no Total area under the curve is 1 \no Curve of the distribution is bell curve \no The curve is symmetric, with half of the values on the left and half of the \nvalues on the right. \nDifference between Poisson and Normal Distribution: \n \nPoisson Normal \nUse Discrete Data Use Continuous Data \nDistribution varies on mean value Symmetric about mean value \nMean = Variance Mean = Median = Mode \n \nStandard normal distribution: \n\uf0b7 Normal distribution with mean = 0 and standard deviation = 1.  \n \n  \n \nProbability distributions are not a Graph. \nA graph is just a visual representation. \n \nThe End',
  'metadata': {'page': 37,
   'source': '..\\data\\pdf\\Statistic_For_Data_Science_1704226429.pdf',
   'creator': 'P

# Integration VectorDB context pipeline With LLM Output

In [ ]:
# creating Simple RAG pipeline with Groq LLM model
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

# using groq API key to initialize llm model
# groq_api_key="May enter grop api directily here"
groq_api_key=os.getenv("GROQ2_API_KEY")

#LLM model 
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant",temperature=0.1,max_tokens=1024)

# creating simple RAG function
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content



In [47]:
answer=rag_simple("What is Normal distribution of the graph?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is Normal distribution of the graph?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 16.37it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


A Normal distribution of the graph is a bell curve that is symmetric about its mean value, with the mean, median, and mode being equal.


In [48]:
answer1=rag_simple("explain importence of statistics?",rag_retriever,llm)
print(answer1)

Retrieving documents for query: 'explain importence of statistics?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 24.51it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


Statistics is crucial as it enables us to collect, analyze, and interpret data, allowing us to make informed decisions and draw conclusions about a population based on a sample. It helps us understand patterns, trends, and relationships within data, which is essential in various fields such as business, medicine, social sciences, and more.


# Advance RAG Pipeline

In [51]:
# --- Advanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("Explain me about z score?", rag_retriever, llm, top_k=5, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:400])

Retrieving documents for query: 'Explain me about z score?'
Top K: 5, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 31.18it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


Answer: A z-score is a measure of how many standard deviations an element is from the mean. It is calculated using the formula:

Z-score = (X - μ) / σ

Where:
- X is the value of the element
- μ is the population mean
- σ is the population standard deviation

In the context of the Z-test, the z-score is used to determine if the sample mean is significantly different from the population mean.
Sources: [{'source': 'Statistic_For_Data_Science_1704226429.pdf', 'page': 21, 'score': 0.21329039335250854, 'preview': 'Page 22 \n \nSH                                                       Statistics for Data Science \n \n  \nP-value: \nThe p-value is defined as the probability of seeing a t-statistic as extreme as the \ncalculated value if the null hypothesis value is true. A low enough p-value is the ground \nfor rejectin...'}, {'source': 'Statistic_For_Data_Science_1704226429.pdf', 'page': 21, 'score': 0.10023480653762817, 'preview': 'We find the Z-statistic of the sample means and calculate th